In [ ]:
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

# 가설검정
from scipy import stats

# 경고문구 무시
import warnings
warnings.filterwarnings("ignore")
# 자연어 처리
# import spacy

## 데이터 정보확인

In [ ]:
df = pd.read_csv('use_df.csv')

In [ ]:
df.head(2)

In [ ]:
df.describe()

In [ ]:
colors = ['black',] * 2
colors[0] = 'crimson'
count = df['Series or Movie'].value_counts()

In [ ]:
# 데이터 시각화 plot.graph_objects 사용
fig = go.Figure(data=[go.Bar(
    x = df["Series or Movie"],
    y = count,
    text = count,
    textposition='auto',
    marker_color=colors 
)])

In [ ]:
# 바차트 제목 및 y축 라벨, x축 라벨 그리기
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'More Movie or Tv Series?',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size= 14),
                  xaxis=dict(
                  title='Category',
                  titlefont_size= 14)
                  )

## viewpoint 정규화
- min-max 정규화 실시

In [ ]:
df['VALUE_std'] = (df['VALUE'] - df['VALUE'].min())/df['VALUE'].max()

- 표준화 실시

In [ ]:
df['VALUE_norm'] = (df['VALUE'] -df['VALUE'].mean())/df['VALUE'].std()

## Tag_count 변수 추가

In [ ]:
len(str(df.Tags.iloc[0]).split(','))

In [ ]:
rows = len(df['Tags'])

In [ ]:
df['Tags'].iloc[0]

In [ ]:
df['Tag_count'] = 0

In [ ]:
for i in range(rows):
    df.Tag_count[i] = len(str(df.Tags.iloc[i]).split(','))

In [ ]:
df.head(1)

In [ ]:
# 정규화 진행

df['Tag_count_norm'] = (df['Tag_count'] - df['Tag_count'].min())/df['Tag_count'].max()  ## 

## movie 데이터와 series 데이터 분리

In [ ]:
df_movie = df[df['Series or Movie']=='Movie']
df_series = df[df["Series or Movie"] == "Series"]

## movie 태그 수와 viewpoint
- 데이터 정리 및 카이제곱 검정(독립성 검사)

In [ ]:
df_movie.head(1)

In [ ]:
df_count = df_movie[['VALUE','Tag_count']]

In [ ]:
df_count.isnull().sum()

In [ ]:
df_count.describe()

### 카이제곱검정 
- 귀무가설 : 두 변수는 연관성이 없다, 차이가 없다. 
    - => tag_count의 증감은 value와의 상관성이 없다 
- 대립가설 : 두 변수는 연관성이 있다, 차이가 있다
    - => tag_count의 증감은 value와의 상관성이 있다

In [ ]:
value = list(df_count.VALUE)
tag_count = list(df_count.Tag_count)

In [ ]:
chis= stats.chisquare(value, tag_count)

In [ ]:
statistic,pvalue = chis
print("static : {}, p-value : {}".format(statistic,pvalue))
if pvalue < 0.05:
    print("신뢰수준하에서 유의한 관계. 대립가설 채택")
else:
    print("신뢰수준하에서 무의미한 관계. 귀무가설 채택")

In [ ]:
df_count.corr()

In [ ]:
plt.scatter(value, tag_count)
plt.show()

 -  카이제곱검정 결과 두 변수들간의 유의수준 0.05에서 tag_count의 증감은 value와의 상관성이 있다고 할 수 있다.
 - 산점도와 상관계수 확인을 해봤을 때 약한 양의 상관성이 있다고 볼 수 있다.

## movie 태그들과 viewpoint
 - 귀무가설 : 태그들은 viewpoint에 영향이 없다
 - 대립가설 : 태그들은 viewpoint에 영향이 있다

### 태그 분할

In [ ]:
df_tag = df_movie[['VALUE','Tags']]

In [ ]:
df_tag.head(1)

In [ ]:
df_tag.info()

In [ ]:
df_tag.set_index('VALUE',inplace=True)

In [ ]:
df_tag.head(10)

In [ ]:
result = df_tag['Tags'].str.split(',')  # 리스트화

In [ ]:
result = result.apply(lambda x: pd.Series(x))

In [ ]:
result.stack()

In [ ]:
result.stack().reset_index(level=1, drop=True)

In [ ]:
result = result.stack().reset_index(level=1, drop=True).to_frame('Tag_single')
result

In [ ]:
rs = result.reset_index()

In [ ]:
rs.head(5)

In [ ]:
len(list(rs.Tag_single.unique()))  # 태그들 종류수

In [ ]:
from copy import copy
exiris = copy(rs)
sns.pairplot(exiris, hue="Tag_single")

### 태그별 카운트 수 확인 및 의미 없는 태그 정리
-  박스플롯의 극값과 1,2개 같은 빈도수가 의미가 없는 태그들을 삭제

In [ ]:
from collections import Counter

In [ ]:
tag_counter = Counter(rs.Tag_single)

In [ ]:
tag_counter = dict(tag_counter)

In [ ]:
dfdf = pd.DataFrame(pd.Series(tag_counter), columns=['counter']) 

In [ ]:
dfdf

In [ ]:
dfdf2 = dfdf[dfdf['counter'] >= 2]
dfdf2 = dfdf2[dfdf2['counter'] <= 20]

In [ ]:
dfdf2.describe()

**태그 수의 2개 이상 20개이하인 로우 값이 256개로 대부분이고, 3분위 수들과 표준 편차를 봤을 때 표준 편차는 4.7 정도, 3분위 수의 차이는 3~4 정도이므로 viewpoint 산정시 영향이 없을 것으로 예상 삭제하였다.**

In [ ]:
dfdf_re = dfdf[dfdf['counter'] > 20]

In [ ]:
plt.figure(figsize=(7,6))
box_plt = dfdf_re.boxplot(column='counter')
plt.show()

In [ ]:
dfdf_re.describe()

- 표준편차가 69인 것에 비해서 3분위 수들과 평균을 확인 했을 때 극값에 영향이 큰것을 예상해 극값을 확인하기 위해 박스플롯을 작성
- **박스플롯에서 outliers를 삭제하기 위해서 백분위 기준 25이상 75 이하인 값만 가져오고 태그들 확인**

In [ ]:
df_set = dfdf[dfdf['counter'] >= 29]
df_set = df_set[df_set['counter'] <= 74]

In [ ]:
df_set.describe()

In [ ]:
plt.figure(figsize=(7,6))
box_plt = df_set.boxplot(column='counter')
plt.show()